In [3]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
from scipy import stats

In [5]:
stocks = pd.read_csv('sp_500_stocks.csv')
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

In [6]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2474439382591,
 'week52high': 191.21,
 'week52low': 130,
 'week52highSplitAdjustOnly': 185.68,
 'week52lowSplitAdjustOnly': 132.37,
 'week52change': 0.026932136236530454,
 'sharesOutstanding': 16720324212,
 'float': 0,
 'avg10Volume': 81377399,
 'avg30Volume': 92325171,
 'day200MovingAvg': 156.08,
 'day50MovingAvg': 149.46,
 'employees': 148244,
 'ttmEPS': 6.13,
 'ttmDividendRate': 0.9018627568926878,
 'dividendYield': 0.005901220914224414,
 'nextDividendDate': '',
 'exDividendDate': '2022-08-03',
 'nextEarningsDate': '2022-10-13',
 'peRatio': 25.52355012161774,
 'beta': 1.2331908785437962,
 'maxChangePercent': 58.09008018301522,
 'year5ChangePercent': 3.204092994728414,
 'year2ChangePercent': 0.3554799275090811,
 'year1ChangePercent': 0.03204589793597328,
 'ytdChangePercent': -0.1451564944149135,
 'month6ChangePercent': -0.0643415738192787,
 'month3ChangePercent': -0.002679766795383596,
 'month1ChangePercent': 0.012818508584067297,
 'day30Cha

In [7]:
data['year1ChangePercent']

0.03204589793597328

In [15]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Symbol'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

In [16]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        new_row = pd.Series([symbol, data[symbol]['quote']['latestPrice'], data[symbol]['stats']['year1ChangePercent'],  'N/A'],  index = my_columns)
        final_dataframe = pd.concat([final_dataframe, new_row.to_frame().T], ignore_index=True)
    
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,MMM,123.2,-0.328439,N/A
1,AOS,52.84,-0.21054,N/A
2,ABT,99.65,-0.221051,N/A
3,ABBV,156.88,0.455904,N/A
4,ABMD,268.98,-0.259361,N/A
...,...,...,...,...
497,YUM,112.59,-0.10359,N/A
498,ZBRA,277.14,-0.489906,N/A
499,ZBH,115.68,-0.262821,N/A
500,ZION,51.77,-0.203966,N/A


In [17]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,OXY,73.07,1.037099,N/A
1,DVN,79.4,0.988147,N/A
2,CF,105.75,0.862166,N/A
3,MCK,393.92,0.853931,N/A
4,MRO,30.83,0.822298,N/A
5,XOM,111.62,0.737399,N/A
6,COP,130.98,0.724385,N/A
7,VRTX,325.88,0.695926,N/A
8,MPC,116.95,0.680871,N/A
9,APA,47.39,0.644688,N/A


In [20]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

Enter the value of your portfolio:200000000
200000000


In [21]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = position_size // final_dataframe['Price'][i]
final_dataframe

/var/folders/sy/r631gy8n4r92m2pzw9tynp8w0000gn/T/ipykernel_95284/3313028142.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_dataframe.loc[i, 'Number of Shares to Buy'] = position_size // final_dataframe['Price'][i]


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,OXY,73.07,1.037099,54742.0
1,DVN,79.4,0.988147,50377.0
2,CF,105.75,0.862166,37825.0
3,MCK,393.92,0.853931,10154.0
4,MRO,30.83,0.822298,129743.0
5,XOM,111.62,0.737399,35835.0
6,COP,130.98,0.724385,30539.0
7,VRTX,325.88,0.695926,12274.0
8,MPC,116.95,0.680871,34202.0
9,APA,47.39,0.644688,84405.0


In [79]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        new_row = pd.Series([symbol, 
                             data[symbol]['quote']['latestPrice'],
                             'N/A',
                             data[symbol]['stats']['year1ChangePercent'],
                             'N/A',
                             data[symbol]['stats']['month6ChangePercent'],
                             'N/A',
                             data[symbol]['stats']['month3ChangePercent'],
                             'N/A',
                             data[symbol]['stats']['month1ChangePercent'],
                             'N/A',
                             'N/A'],
                            index = hqm_columns)
        hqm_dataframe = pd.concat([hqm_dataframe, new_row.to_frame().T], ignore_index=True)

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MMM,123.8,N/A,-0.339122,N/A,-0.192101,N/A,-0.110795,N/A,0.048895,N/A,N/A
1,AOS,52.01,N/A,-0.208624,N/A,-0.19219,N/A,-0.112821,N/A,0.047604,N/A,N/A
2,ABT,100.88,N/A,-0.221058,N/A,-0.176487,N/A,-0.094569,N/A,-0.023425,N/A,N/A
3,ABBV,160.09,N/A,0.455454,N/A,-0.023713,N/A,0.007691,N/A,0.05823,N/A,N/A
4,ABMD,280.73,N/A,-0.265307,N/A,-0.130384,N/A,-0.060146,N/A,0.079845,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
497,YUM,113.47,N/A,-0.10252,N/A,-0.074819,N/A,-0.046576,N/A,0.005499,N/A,N/A
498,ZBRA,282.67,N/A,-0.485502,N/A,-0.301291,N/A,-0.157776,N/A,0.0249,N/A,N/A
499,ZBH,112.57,N/A,-0.267923,N/A,-0.134159,N/A,0.029085,N/A,0.04665,N/A,N/A
500,ZION,51.35,N/A,-0.204855,N/A,-0.149418,N/A,-0.042035,N/A,-0.055628,N/A,N/A


In [80]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100

# Print each percentile score to make sure it was calculated properly
#for time_period in time_periods:
 #   print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_dataframe

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/scipy/stats/_stats_py.py:112: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  warnings.warn("The input array could not be properly "
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/scipy/stats/_stats_py.py:112: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  warnings.warn("The input array could not be properly "
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/scipy/stats/_stats_py.py:112: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  warnings.warn("The input array could not be properly "
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/scipy/stats/_stats_py.py:112: RuntimeWarning: The input array could not be properly checked for nan v

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MMM,123.8,N/A,-0.339122,0.173307,-0.192101,0.278884,-0.110795,0.26494,0.048895,0.507968,N/A
1,AOS,52.01,N/A,-0.208624,0.378486,-0.19219,0.276892,-0.112821,0.258964,0.047604,0.501992,N/A
2,ABT,100.88,N/A,-0.221058,0.364542,-0.176487,0.318725,-0.094569,0.314741,-0.023425,0.201195,N/A
3,ABBV,160.09,N/A,0.455454,0.948207,-0.023713,0.699203,0.007691,0.62749,0.05823,0.537849,N/A
4,ABMD,280.73,N/A,-0.265307,0.298805,-0.130384,0.436255,-0.060146,0.418327,0.079845,0.645418,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
497,YUM,113.47,N/A,-0.10252,0.577689,-0.074819,0.581673,-0.046576,0.460159,0.005499,0.302789,N/A
498,ZBRA,282.67,N/A,-0.485502,0.063745,-0.301291,0.093625,-0.157776,0.13745,0.0249,0.384462,N/A
499,ZBH,112.57,N/A,-0.267923,0.292829,-0.134159,0.422311,0.029085,0.673307,0.04665,0.496016,N/A
500,ZION,51.35,N/A,-0.204855,0.38247,-0.149418,0.384462,-0.042035,0.474104,-0.055628,0.121514,N/A


In [81]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MMM,123.8,N/A,-0.339122,0.173307,-0.192101,0.278884,-0.110795,0.26494,0.048895,0.507968,0.306275
1,AOS,52.01,N/A,-0.208624,0.378486,-0.19219,0.276892,-0.112821,0.258964,0.047604,0.501992,0.354084
2,ABT,100.88,N/A,-0.221058,0.364542,-0.176487,0.318725,-0.094569,0.314741,-0.023425,0.201195,0.299801
3,ABBV,160.09,N/A,0.455454,0.948207,-0.023713,0.699203,0.007691,0.62749,0.05823,0.537849,0.703187
4,ABMD,280.73,N/A,-0.265307,0.298805,-0.130384,0.436255,-0.060146,0.418327,0.079845,0.645418,0.449701
...,...,...,...,...,...,...,...,...,...,...,...,...
497,YUM,113.47,N/A,-0.10252,0.577689,-0.074819,0.581673,-0.046576,0.460159,0.005499,0.302789,0.480578
498,ZBRA,282.67,N/A,-0.485502,0.063745,-0.301291,0.093625,-0.157776,0.13745,0.0249,0.384462,0.169821
499,ZBH,112.57,N/A,-0.267923,0.292829,-0.134159,0.422311,0.029085,0.673307,0.04665,0.496016,0.471116
500,ZION,51.35,N/A,-0.204855,0.38247,-0.149418,0.384462,-0.042035,0.474104,-0.055628,0.121514,0.340637


In [82]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False, inplace = True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(drop = True, inplace = True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,SLB,52.4,N/A,0.562945,0.974104,0.361803,0.99004,0.47185,0.998008,0.502552,1.0,0.990538
1,DVN,78.31,N/A,1.004901,0.998008,0.384908,0.992032,0.317904,0.984064,0.317434,0.986056,0.99004
2,COP,126.72,N/A,0.7124,0.988048,0.413117,0.996016,0.387346,0.996016,0.261091,0.968127,0.987052
3,MRO,31.16,N/A,0.794003,0.992032,0.237979,0.968127,0.292308,0.978088,0.367387,0.992032,0.98257
4,HES,142.4,N/A,0.56279,0.972112,0.344239,0.986056,0.290823,0.976096,0.302132,0.982072,0.979084
5,MPC,118.77,N/A,0.704481,0.986056,0.345704,0.988048,0.275705,0.972112,0.225387,0.960159,0.976594
6,APA,47.18,N/A,0.650448,0.982072,0.17308,0.940239,0.322991,0.986056,0.384501,0.994024,0.975598
7,FANG,160.1,N/A,0.457611,0.954183,0.255446,0.972112,0.313726,0.982072,0.346871,0.99004,0.974602
8,XOM,112.34,N/A,0.748155,0.99004,0.323487,0.984064,0.195782,0.936255,0.242282,0.962151,0.968127
9,EOG,135.6,N/A,0.529973,0.968127,0.229798,0.960159,0.285971,0.974104,0.224181,0.958167,0.965139


In [83]:
portfolio_input()

Enter the value of your portfolio:
That's not a number! 
 Try again:
Enter the value of your portfolio:1000000


In [84]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = position_size // hqm_dataframe['Price'][i]
hqm_dataframe

/var/folders/sy/r631gy8n4r92m2pzw9tynp8w0000gn/T/ipykernel_95284/2514460762.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hqm_dataframe.loc[i, 'Number of Shares to Buy'] = position_size // hqm_dataframe['Price'][i]


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,SLB,52.4,381.0,0.562945,0.974104,0.361803,0.99004,0.47185,0.998008,0.502552,1.0,0.990538
1,DVN,78.31,255.0,1.004901,0.998008,0.384908,0.992032,0.317904,0.984064,0.317434,0.986056,0.99004
2,COP,126.72,157.0,0.7124,0.988048,0.413117,0.996016,0.387346,0.996016,0.261091,0.968127,0.987052
3,MRO,31.16,641.0,0.794003,0.992032,0.237979,0.968127,0.292308,0.978088,0.367387,0.992032,0.98257
4,HES,142.4,140.0,0.56279,0.972112,0.344239,0.986056,0.290823,0.976096,0.302132,0.982072,0.979084
5,MPC,118.77,168.0,0.704481,0.986056,0.345704,0.988048,0.275705,0.972112,0.225387,0.960159,0.976594
6,APA,47.18,423.0,0.650448,0.982072,0.17308,0.940239,0.322991,0.986056,0.384501,0.994024,0.975598
7,FANG,160.1,124.0,0.457611,0.954183,0.255446,0.972112,0.313726,0.982072,0.346871,0.99004,0.974602
8,XOM,112.34,178.0,0.748155,0.99004,0.323487,0.984064,0.195782,0.936255,0.242282,0.962151,0.968127
9,EOG,135.6,147.0,0.529973,0.968127,0.229798,0.960159,0.285971,0.974104,0.224181,0.958167,0.965139


In [85]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

In [86]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [87]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', percent_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)


In [88]:
writer.save()

/var/folders/sy/r631gy8n4r92m2pzw9tynp8w0000gn/T/ipykernel_95284/934276808.py:1: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
